In [ ]:
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from simbio import Simulator

from mito import ARM, ARM_Cito, Mitochondria
from n_mito.loop_simulator import LoopSimulator

In [ ]:
class Fig:
    def __init__(self, *, dlim=None, shifted: bool = False):
        self.shifted = shifted

        self.fig = plt.figure(figsize=(12, 4))
        subfigs = self.fig.subfigures(ncols=2, width_ratios=[2, 1])
        self.axes = subfigs[0].subplots(2, 2, sharex="col", sharey="row")
        self.ax = subfigs[1].subplots()

        for ax in self.axes[0]:
            ax.secondary_xaxis(
                "top",
                functions=(
                    lambda x: x / 3600,
                    lambda x: x * 3600,
                ),
            ).set(xlabel="time [hours]")

        if dlim is not None:
            match dlim:
                case int():
                    xlim = ylim = (-dlim, dlim)
                case (int(), int()):
                    xlim = ylim = dlim
                case [(int(), int()), (int(), int())]:
                    xlim, ylim = dlim
                case _:
                    raise ValueError(f"{dlim=}")

            self.ax.set(xlim=xlim, ylim=ylim)
        self.ax.grid()
        self.ax.axhline(0, color="black")
        self.ax.axvline(0, color="black")

    def plot(
        self,
        df: pd.DataFrame,
        *,
        normalize: bool = False,
        color=None,
        label=None,
        **kwargs,
    ):
        if normalize:
            df = df.transform(lambda x: x / x.max())
            self.axes[0, 0].set_ylim(-0.1, 1.1)

        diff = df.diff()
        max_diff = diff.idxmax()
        delta_diff = (max_diff - max_diff["cytoplasm.C3_A"]) / 60

        def shifted(df: pd.DataFrame):
            df = df.copy()
            df.index = df.index - max_diff["cytoplasm.C3_A"]
            return df

        df.plot(ax=self.axes[0, 0], legend=False, color=color, **kwargs)
        diff.plot(
            ax=self.axes[1, 0],
            legend=False,
            color=color,
            **kwargs,
            xlabel="time [seconds]",
        )

        shifted(df).plot(ax=self.axes[0, 1], legend=False, color=color, **kwargs)
        shifted(diff).plot(
            ax=self.axes[1, 1],
            legend=False,
            color=color,
            **kwargs,
            xlabel="time [seconds]",
        )

        self.ax.scatter(
            delta_diff["cytoplasm.C8_A"],
            delta_diff["cytoplasm.Apop"],
            color=color,
            label=label,
        )
        return self

## ARM 1

In [ ]:
sim = Simulator(ARM, backend="numba")
t = np.linspace(0, 30_000, 1000)

cols = [
    str(ARM.cytoplasm.C3_A),
    str(ARM.cytoplasm.C8_A),
    str(ARM.cytoplasm.Apop),
]

### Default

In [ ]:
df_1_ext = sim.solve(
    save_at=t,
    values={ARM.L_concentration: 1000, ARM.IntrinsicStimuli_concentration: 0},
)[cols]

df_1_int = sim.solve(
    save_at=t,
    values={ARM.L_concentration: 0, ARM.IntrinsicStimuli_concentration: 100},
)[cols]

(
    Fig(dlim=20, shifted=True)
    .plot(df_1_ext, color="C0", label="extrinsic")
    .plot(df_1_int, color="C1", label="intrinsic")
)

In [ ]:
from traitlets import validate


class DisabledFloatSlider(ipywidgets.FloatLogSlider):
    @validate("value")
    def _validate_value(self, proposal):
        """Cap and floor value"""
        if self.disabled:
            return 0
        value = proposal["value"]
        if self.base**self.min > value or self.base**self.max < value:
            value = min(max(value, self.base**self.min), self.base**self.max)
        return value


button = ipywidgets.Button(description="extrinsic / intrinsic")
extrinsic = DisabledFloatSlider(
    value=0, min=-3, max=5, description="extrinsic", disabled=True
)
intrinsic = DisabledFloatSlider(value=100, min=-3, max=5, description="intrinsic")


@button.on_click
def switch(_, /):
    extrinsic.disabled = not extrinsic.disabled
    intrinsic.disabled = not intrinsic.disabled
    if extrinsic.disabled:
        extrinsic.value = 0
    if intrinsic.disabled:
        intrinsic.value = 0

In [ ]:
def plot(df):
    (
        Fig(dlim=100)
        .plot(df_1_ext[cols], color="C0", label="extrinsic")
        .plot(df_1_int[cols], color="C1", label="intrinsic")
        .plot(df[cols], color="C2")
    )


display(button)
sim.interact(
    save_at=t,
    values={
        ARM.mitocondria_volume_fraction: ipywidgets.FloatLogSlider(
            value=0.07, min=-3, max=1, description="Mito volume"
        ),
        ARM.L_concentration: extrinsic,
        ARM.IntrinsicStimuli_concentration: intrinsic,
        ARM.mitocondria.pore_transport_rate: ipywidgets.FloatLogSlider(
            value=10, min=-2, max=3, description="pore"
        ),
        ARM.mitocondria.transloc_rate: ipywidgets.FloatLogSlider(
            value=1e-2, min=-4, max=0, description="transloc"
        ),
    },
    func=plot,
)

In [ ]:
def plot(df):
    Fig(dlim=100).plot(df_1_ext[cols], color="C0").plot(df[cols], color="C1")


sim.interact(
    save_at=t,
    values={
        ARM.volume: ipywidgets.FloatLogSlider(value=1, min=-2, max=1),
        ARM.mitocondria_volume_fraction: ipywidgets.FloatLogSlider(
            value=0.07, min=-3, max=1
        ),
        ARM.L_concentration: (0, 0, 1),
        ARM.IntrinsicStimuli_concentration: ipywidgets.FloatLogSlider(
            value=100, min=-3, max=5
        ),
        # ARM.mitocondria.pore_transport_rate: ipywidgets.FloatLogSlider(
        #     value=10, min=-2, max=3
        # ),
        # ARM.mitocondria.transloc_rate: ipywidgets.FloatLogSlider(
        #     value=1e-2, min=-4, max=0
        # ),
    },
    func=plot,
)

## Some graphs

In [ ]:
from matplotlib.cm import viridis

fig = Fig(dlim=100)

volumes = [0.01, 0.07, 0.3]
for color, v in zip(viridis.resampled(len(volumes)).colors, volumes):
    df = sim.solve(save_at=t, values={ARM.mitocondria_volume_fraction: v})
    fig.plot(df[cols], color=color, label=v)

fig.ax.legend(title="volume fraction")

In [ ]:
from matplotlib.cm import viridis

fig = Fig(dlim=100)

volumes = [0.01, 0.07, 0.3, 1]
for color, v in zip(viridis.resampled(len(volumes)).colors, volumes):
    df = sim.solve(
        save_at=t,
        values={
            ARM.mitocondria_volume_fraction: v,
            ARM.L_concentration: 0,
            ARM.IntrinsicStimuli_concentration: 100,
        },
    )
    fig.plot(df[cols], color=color, label=v)

fig.ax.legend(title="volume fraction")

## ARM N

In [ ]:
loop_sim = LoopSimulator(
    ARM_Cito,
    Mitochondria(
        CytoC_C=ARM_Cito.CytoC_C,
        Smac_C=ARM_Cito.Smac_C,
        Bax_A=ARM_Cito.Bax_A,
    ),
    [
        ARM_Cito.CytoC_C,
        ARM_Cito.Smac_C,
        ARM_Cito.Bax_A,
    ],
)

In [ ]:
def get_values(
    *,
    L,
    intrinsic,
    N: int,
    volume: float,
    global_mitocondria_volume_fraction: float,
):
    return (
        {
            ARM_Cito.volume: volume,
            ARM_Cito.L: L * volume,
            ARM_Cito.IntrinsicStimuli: intrinsic * volume,
        },
        {
            Mitochondria.volume: volume * global_mitocondria_volume_fraction / N,
            Mitochondria.Bax4: np.zeros(N),
        },
    )

In [ ]:
display(button)


@ipywidgets.interact(
    N=ipywidgets.FloatLogSlider(value=1, min=0, max=2, continuous_update=False),
    volume_cell=ipywidgets.FloatLogSlider(
        value=1, min=-3, max=3, continuous_update=False
    ),
    global_mitocondria_volume_fraction=ipywidgets.FloatLogSlider(
        value=0.07, min=-3, max=3, continuous_update=False
    ),
    L=extrinsic,
    intrinsic=intrinsic,
)
def _(
    N: int,
    volume_cell: float,
    global_mitocondria_volume_fraction: float,
    L,
    intrinsic,
):
    N = max(int(N), 1)
    main_values, loop_values = get_values(
        L=L if L > 1 else 0,
        intrinsic=intrinsic if intrinsic > 1 else 0,
        N=N,
        volume=volume_cell,
        global_mitocondria_volume_fraction=global_mitocondria_volume_fraction,
    )
    df = loop_sim.solve(
        save_at=t,
        main_values=main_values,
        loop_values=loop_values,
    )

    columns = [
        ARM_Cito.C3_A.variable,
        ARM_Cito.C8_A.variable,
        ARM_Cito.Apop.variable,
    ]
    df = df[columns].rename(columns={k: f"cytoplasm.{k}" for k in columns})

    (
        Fig(dlim=100)
        .plot(df_1_ext, color="C0", label="extrinsic")
        .plot(df_1_int, color="C1", label="intrinsic")
        .plot(df, color="C2")
        .fig.suptitle(N)
    )

In [ ]:
def run(*, N: int, global_volume: float):
    main_values, loop_values = get_values(
        L=0,
        intrinsic=100,
        N=N,
        volume=1,
        global_mitocondria_volume_fraction=global_volume,
    )
    df = loop_sim.solve(
        save_at=t,
        main_values=main_values,
        loop_values=loop_values,
    )

    columns = [
        ARM_Cito.C3_A.variable,
        ARM_Cito.C8_A.variable,
        ARM_Cito.Apop.variable,
    ]
    df = df[columns].rename(columns={k: f"cytoplasm.{k}" for k in columns})
    return df


from matplotlib.cm import viridis

fig = Fig(dlim=100)

volumes = [0.01, 0.07, 0.3, 1]
for color, v in zip(viridis.resampled(len(volumes)).colors, volumes):
    for N, ls in {1: "-", 100: "--"}.items():
        df = run(N=N, global_volume=v)
        fig.plot(df, color=color, label=f"{N=} {v=}", linestyle=ls)

fig.ax.legend(title="volume fraction", bbox_to_anchor=(1, 1))